In [0]:
import keras
import numpy as np
import pandas as pd
import cv2
from keras import backend as K
from keras.layers.merge import concatenate
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten,Input,concatenate
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D, LocallyConnected1D
from keras.utils import np_utils
#from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam, SGD, RMSprop, Adagrad
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import itertools

In [10]:
data = pd.read_csv("https://raw.githubusercontent.com/Moshiur26/Datasets/master/Promoter/New%20Dataset/promoter1234.csv",header=None)
data=np.asarray(data)
data = np.delete(data, (0), axis=0)
data = np.delete(data, (0), axis=1)
X=data[:,:data.shape[1]-1]
y=data[:,-1:]
y = np.unique(y, return_inverse=True)[1].tolist()
ys=y
X = to_categorical(X)
y = to_categorical(y)
#X = X.reshape(5720, 81, 1)
X.shape


(5720, 81, 4)

In [0]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=0.15)

In [12]:
#LocallyConnected1D
k=4
visible1 = Input(shape=(81,4))
model11=Conv1D(81, kernel_size=k, activation='relu')(visible1)
#drop11=Dropout(0.2)(model11)
pool11=MaxPooling1D(pool_size=3)(model11)#(drop11)
con11=Conv1D(64, kernel_size=k, activation='relu')(pool11)
drop12=Dropout(0.3)(con11)
pool12=MaxPooling1D(pool_size=3)(drop12)
con12=Conv1D(32, kernel_size=k, activation='relu')(pool12)
drop13=Dropout(0.4)(con12)
con13=Conv1D(16, kernel_size=k, activation='relu')(drop13)
drop14=Dropout(0.6)(con13)

#on14=Conv1D(10,kernel_size=k, activation='relu')(drop14)
fltn1=Flatten()(drop14)

k=3
visible2 = Input(shape=(81,4))
model21=Conv1D(81, kernel_size=k, activation='relu')(visible2)
drop21=Dropout(0.2)(model21)
pool21=MaxPooling1D(pool_size=3)(drop21)
con21=Conv1D(64, kernel_size=k, activation='relu')(pool21)
drop21=Dropout(0.2)(con21)
pool22=MaxPooling1D(pool_size=3)(drop21)
con22=Conv1D(32, kernel_size=k, activation='relu')(pool22)
drop22=Dropout(0.4)(con22)
con23=Conv1D(16, kernel_size=k, activation='relu')(drop22)
drop23=Dropout(0.6)(con23)
fltn2=Flatten()(drop23)

k=6
visible3 = Input(shape=(81,4))
model31=Conv1D(81, kernel_size=k, activation='relu')(visible3)
drop31=Dropout(0.2)(model31)
pool31=MaxPooling1D(pool_size=2)(drop31)
con31=Conv1D(64, kernel_size=k, activation='relu')(pool31)
drop32=Dropout(0.4)(con31)
pool32=MaxPooling1D(pool_size=2)(drop32)
con32=Conv1D(32, kernel_size=k, activation='relu')(pool32)
drop33=Dropout(0.6)(con32)
fltn3=Flatten()(drop33)



marge=concatenate([fltn1, fltn2, fltn3])
#model.add(concatenate([fltn1, fltn2]))
dns0=Dense(16, activation='relu')(marge)
dns1=Dense(8, activation='relu')(dns0)
dns2=Dense(2, activation='softmax')(dns1)

model = Model(input=[visible1, visible2, visible3], output=dns2)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 81, 4)        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 81, 4)        0                                            
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 79, 81)       1053        input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 78, 81)       1377        input_4[0][0]                    
__________________________________________________________________________________________________
dropout_14

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=22,batch_size=50, verbose=1)
#model.fit([X_train,X_train], y_train,validation_data=([X_val,X_val], y_val),nb_epoch=10, batch_size=28)
model.fit([X_train,X_train,X_train], y_train,validation_data=([X_val,X_val,X_val], y_val),epochs=22,batch_size=50, verbose=1)

Train on 4132 samples, validate on 730 samples
Epoch 1/22
4132/4132 [==============================] - 8s 2ms/step - loss: 0.6019 - acc: 0.6692 - val_loss: 0.4979 - val_acc: 0.7849
Epoch 2/22
4132/4132 [==============================] - 6s 1ms/step - loss: 0.5204 - acc: 0.7556 - val_loss: 0.4919 - val_acc: 0.7836
Epoch 3/22
4132/4132 [==============================] - 6s 1ms/step - loss: 0.4981 - acc: 0.7650 - val_loss: 0.4650 - val_acc: 0.8000
Epoch 4/22
4132/4132 [==============================] - 6s 1ms/step - loss: 0.4824 - acc: 0.7795 - val_loss: 0.5057 - val_acc: 0.7616
Epoch 5/22
4132/4132 [==============================] - 6s 1ms/step - loss: 0.4686 - acc: 0.7921 - val_loss: 0.4325 - val_acc: 0.8137
Epoch 6/22
4132/4132 [==============================] - 6s 1ms/step - loss: 0.4470 - acc: 0.8064 - val_loss: 0.4150 - val_acc: 0.8260
Epoch 7/22
4132/4132 [==============================] - 6s 1ms/step - loss: 0.4392 - acc: 0.8035 - val_loss: 0.4020 - val_acc: 0.8315
Epoch 8/22
4132

In [14]:
y_pred = model.predict([X_test,X_test,X_test])
print(y_pred.shape)

y_true = np.argmax(y_test, axis=1)
print(y_true.shape)
y_pred = np.argmax(y_pred, axis=1)

(858, 2)
(858,)


In [15]:
print(accuracy_score(y_true, y_pred) * 100)
print(classification_report(y_true, y_pred))

84.96503496503497
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       428
           1       0.84      0.86      0.85       430

   micro avg       0.85      0.85      0.85       858
   macro avg       0.85      0.85      0.85       858
weighted avg       0.85      0.85      0.85       858

